In [ ]:
"""
The way the new evaluation works is following this simple algorithm:

if (sentence contains key words):
  We want a compact/'specific' explanation, where only the key words are highlighted
else:
  We want a general/'nuanced' explanations, where all words have similar relevance


How to we check for key words:
 - fidelity: words that flip the prediction are more likely to be key words
 - realism: words that are similar to a set of pre-identified key words
"""

In [ ]:
import numpy as np
import copy
from nltk.corpus import wordnet as wn

In [ ]:
sentence = [""]
x = [0]
model = lambda x: 0
keyWordsClass0 = [] # pre-identified set of key words for class 0
keyWordsClass1 = [] # pre-identified set of key words for class 1
explanation = []

In [ ]:
def realism(sentence, positive_words, negative_words, prediction, num_words):
    """
    sentence: a list of words
    positive words: words associated with class 1
    negative words: words associated with class 0
    prediction: the class of the prediction

    returns the indices of the realistic words, if there are any
    """
    keywords = set()
    for (i, word) in enumerate(sentence):
        if (i >= num_words):
            return keywords
        if (prediction == 0):
            for nw in negative_words:
                try:
                    if (wn.synsets(word)[0].wup_similarity(wn.synsets(nw)[0]) > 0.75):
                        keywords.add((i, word))
                except:
                    if (word == nw):
                        keywords.add((i, word))
        if (prediction == 1):
            for pw in positive_words:
                try:
                    if (wn.synsets(word)[0].wup_similarity(wn.synsets(pw)[0]) > 0.75):
                        keywords.add((i, word))
                except:
                    if (word == pw):
                        keywords.add((i, word))
    return keywords

In [ ]:
def fidelity(x, model, num_words):
    """
    x: the input words
    model: the prediction model
    """
    keywords = []
    samples = 5
    for (i, w) in enumerate(x):
        if (i >= num_words):
            return keywords
        if (w != 0):
            change_rating = 0
            for change in range(samples):
                new_x = copy.deepcopy(x)
                new_x[i] = new_x[i] + (100 * change / samples)
                if (model(np.array([x])) != model(np.array([new_x]))):
                    change_rating += 1/samples

            if (change_rating > samples * 3 / 4):
                keywords.append((i, w))

    return keywords

In [ ]:
def compactness(keywords, explanation, num_words):
    """
    explanation: the feature attributions
    """
    explanation = explanation[:num_words]
    q0, q4 = np.percentile(explanation, [0, 100])
    iqr = q4 - q0
    upper_bound = q4 - (0.4 * iqr)
    outliers = len([x for x in explanation if x > upper_bound])
    return outliers

In [ ]:
def correctness(keywords, explanation, num_words):
    """
    keywords: the list of keywords + their indices
    explanation: the feature attributions
    """
    explanation = explanation[:num_words]
    q0, q4 = np.percentile(explanation, [0, 100])
    iqr = q4 - q0
    upper_bound = q4 - (0.4 * iqr)
    result = 0
    for (index, word) in keywords:
        if explanation[index] > upper_bound:
            result += 1
    return result

In [ ]:
def nuance(explanation, num_words):
    """
    explanation: the feature attributions
    """
    explanation = explanation[:num_words]
    if (np.max(explanation) == np.min(explanation)):
        return 1
    return 1 - np.std(explanation)/np.std([np.max(explanation), np.min(explanation)])

In [ ]:
def evaluate(words, ids, model, positive_words, negative_words, explanation):
    num_words = np.sum(np.array(words) != "[PAD]")
    keywords = []
    keywords += realism(words, positive_words, negative_words, model(np.array([ids])), num_words)
    fid_words = fidelity(ids, model, num_words)
    keywords += fid_words
    print("fidelity words:", fid_words)
    quality = None
    print("Keywords", keywords)
    print(num_words)

    if len(keywords) > 0:
        if (compactness(keywords, explanation, num_words) == 0):
            quality = 0
        else:
            quality = correctness(keywords, explanation, num_words) * correctness(keywords, explanation, num_words) / compactness(keywords, explanation, num_words) / len(keywords)
    else:
        quality = nuance(explanation, num_words)

    print("Compactness", compactness(keywords, explanation, num_words))
    print("Correctness", correctness(keywords, explanation, num_words))
    print("Nuance", nuance(explanation, num_words))
    return quality